### Code to process multichannel Tiff files of the 3D model 201709 into single channel tiffs

In [ ]:
import sys
from tifffile import imread, imsave
import os, re
import numpy as np
import csv
import cv2
import napari
import pandas as pd


Some of the input files of the input multichannel tiffs of the model 201710 require stitching because the IMC acquisition stopped. In these cases, once the measuring was restarted few rows were reacquired causing the second image to have empty pixel values. Thus, for stitiching some empty rows had to be cut from the second image. 

**This code assumes that the channel order is the same for all the acquired slices.** 

In [ ]:
def change_img_size_singleTiffs(img_i, max_dimensions):
    
        if len(img_i.shape) == 2:
            xysize = img_i.shape

        else:
            return "It should be a 2D array per one channel"

        if len(max_dimensions) == 2:
            ymax = max_dimensions[1]
            xmax = max_dimensions[0]

        else:
            return "Padding should have input as list of two arguments:maximum image size mxn\
                    in x (columns of matrix) and in y (rows of matrix) "

        img_i= np.pad(img_i,[(0,ymax-xysize[0]), (0,xmax-xysize[1])], 'constant')
        
        return img_i

In [ ]:
def remove_str_from_end(astring, trailing):
    if astring.endswith(trailing):
        return astring[:-len(trailing)]
    return astring

**This function rotates images and the expected interpolation method has to be set inside this function: currently set to cubic: a bicubic interpolation over 4×4 pixel neighborhood***
- INTER_NEAREST – a nearest-neighbor interpolation
- INTER_LINEAR – a bilinear interpolation (used by default)
- INTER_AREA – resampling using pixel area relation
- INTER_CUBIC – a bicubic interpolation over 4×4 pixel neighborhood
- INTER_LANCZOS4 – a Lanczos interpolation over 8×8 pixel neighborhood

In [ ]:
#code taken from  https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
def rotateImage(image, angle, dsize = None):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
     
    # perform the actual rotation and return the image
    result = cv2.warpAffine(image, M, (nW, nH), dsize,flags=cv2.INTER_CUBIC)   
        
    return result

In [ ]:
def get_image_stack_for_one_channel(stack_folder, target_metal, csv_dict):    
    """Get images for a chosen channel from single channel tiffs, 
    read the folder order from a csv file to get the 3D stack in the right order"""
    
    single_channel_list = []
    # need to sort the files in a stack:    
    stack_dir = get_folder_order_from_file(stack_folder,csv_dict)
    
    for e in stack_dir:
        met_dir = os.path.join(stack_folder, e)
        channel_files = os.listdir(met_dir)

        channel = [x for x in channel_files if target_metal in x]

        if len(channel) == 1:
            im_path = os.path.join(met_dir, channel[0])
            single_channel_list.append(im_path)

        else:
            print("Only one file per channel allowed, check if the search string is correct")

    return single_channel_list


def get_folder_order_from_file(stack_dir, csv_dict):
    list_len = len(csv_dict)
    stack_list = [None] *list_len
    files_dir = os.listdir(stack_dir)
    
    for item in files_dir:
        order_index = csv_dict[item]
        stack_list[order_index] = item

    return stack_list

Set input folders

In [ ]:
#convert omeTIFFS into single channel tiffs required for registration
#first check the size of the images and make them same size for single channel tiffs
# find images that require rotation
# find images that require stitching

# Input: folder for tiffs, 
img_folder_initial = '~/201709'

# Output folder
single_tiff_folder = '~/3Dstack_single_channel_tiffs/'

#CSV files with two columns and no header: file name, file order as number
csv_stack_order='~/image_order_hypoxia_model201709.csv'
csv_channel_order='~/channel_order_hypoxia_model201709.csv'
# final csv with file order after images that required stitching were merged
stack3d_order = '~/final_3D_stack_order_hypoxia_model201709.csv'

csv_pannel = '~/hypoxia_model_201709_panel.csv'
csv_pannel_singleTIFF = 'singleTIFFs'
pannel = pd.read_csv(csv_pannel)


In [ ]:
images = {}
reader = csv.reader(open(csv_stack_order, 'r'))    
for k,v in reader:
    v= int(v)
    images[v] = k
        
channels = {}
reader = csv.reader(open(csv_channel_order, 'r'))    
for k,v in reader:
    v= int(v)
    channels[v] = k

final3d_order = {}
reader = csv.reader(open(stack3d_order, 'r'))    
for k,v in reader:
    v= int(v)
    final3d_order[k] = v

if pannel.shape[1] > 1:
        selected = pannel[csv_pannel_singleTIFF]
        metalcolumn = 'Metal Tag'
        selected_metals = [str(n) for s, n in zip(selected, pannel[metalcolumn]) if s]    

Uncomment this following cell to determin the largest size of the image in the 3D stack (has to be run only once):

In [ ]:
#check imgae size accross all images for all the slices
img_dimy= []
img_dimx = []
for key in sorted(images.keys()):    
    im_name = str(images[key])

    if not im_name.endswith('.tiff'):
        continue  
    im = imread(os.path.join(img_folder_initial,im_name))

    img_dim = im.shape
    img_dimy.append(im.shape[1])
    img_dimx.append(im.shape[2])
    
row_max=max(img_dimy)
col_max= max(img_dimx)

print(row_max, col_max)

In [ ]:
#set from the previous cell:
row_max = 1150
col_max =1040
max_dimensions = [col_max, row_max]

The following cell stitches and rotates the images, and saves the output images as single channel tiffs. The following first 5 channels from the original images are not saved: Start_push, End_push, Pushes_duration, X,Y,Z

In [ ]:
#key values taken from the csv file called image_order_model201710.csv
needs_stitching = [(51,52), (53,54)]

#number of empty rows removed for the images
remove_row_pixels = {51:'14', 53:'25'}

#some images are off-set that has to be removed: ie translation: +ve will shift left, -ve shifts right& adds padding
offset_pixels = {51:'23', 53:'34'}

#from the need_stitching tuples the values are added to this list after stiching is completed
alredy_stitched = []

for key in sorted(images.keys()): 
    print(key)
    if [item for item in needs_stitching if key in item]:
        
        if key not in alredy_stitched:
            #retrieve the image name from dictionary based on the name-key pairs
            im_name1 = str(images[key])
            pair = [y[0] for y in needs_stitching].index(key)
            
            #retrieve the second paired image
            key2 = needs_stitching[pair][1]            
            im_name2 = str(images[key2])
            
            #read the tiff stack of broken slices
            im1_ori = imread(os.path.join(img_folder_initial,im_name1))
            im2_ori = imread(os.path.join(img_folder_initial,im_name2))

            #modify file names to create new folder
            im_name_new = remove_str_from_end(im_name1, '.tiff')
            im_name2_new = remove_str_from_end(im_name2, '.tiff')
            im_name_new = im_name_new.replace(" ", "_")
            im_name2_new= im_name2_new.replace(" ", "_")
            new_joint_folder = im_name_new + '_' + im_name2_new
            singleTiff_folder_name = os.path.join(single_tiff_folder,new_joint_folder)
            new_joint_folder = new_joint_folder.replace(" ", "_")
                               
            #creat new folder for each image that will contain single channel tiffs
            if not os.path.exists(singleTiff_folder_name):
                os.mkdir(singleTiff_folder_name)
                
            #set values for image modification based on the dictionary key
            remove_rows =int(remove_row_pixels[key])               
            offset =int(offset_pixels[key])  
            
            print(im1_ori.shape, im2_ori.shape)
            #check that number of channels is the same for both of the images
            if im1_ori.shape[0] == im2_ori.shape[0]:
                
                for i in range(6, im1_ori.shape[0]): 

                    # new_single_tiff_name = im_name + channel_name
                    channel_name = str(channels[i])
                                   
                    if channel_name in selected_metals:
                                                     
                        im1 = im1_ori[i,:,:]
                        im2 = im2_ori[i,:,:]
                        im_list = [im1, im2]

                        #need to find which image requires padding in x dimensions (number of columns)
                        im_col_size = [im1.shape[1], im2.shape[1]]
                        col_padding = abs(im1.shape[1]-im2.shape[1])
                        col_size = min(im_col_size)

                        #return image with index of smallest x value
                        im_index = im_col_size.index(col_size)
                        #print(im_col_size, col_padding)

                        #which image needs extra padding 
                        if im_index == 0:
                            offset1 = offset

                            if offset<0:
                                offset1 = -offset1
                                top_im_to_add = np.pad(im2,((0,0), (offset1,0)), 'constant')
                                top_im_to_add = top_im_to_add[remove_rows:,:]                       

                            else:

                                top_im_to_add = im2[remove_rows:,offset1:] #here cut the top images at x by offset

                            if key == 51:

                                top_im_to_add  = top_im_to_add[:,13:]
                                top_im_to_add= np.pad(top_im_to_add,((0,0), (0,13)), 'constant')
                                img_stitched = np.concatenate((im1,top_im_to_add), axis=0)

                            else:

                                im_new_padded= np.pad(im1,((0,0), (0,col_padding-offset1)), 'constant')
                                img_stitched = np.concatenate((im_new_padded[:-1,:],top_im_to_add), axis=0)


                        if im_index == 1:

                            offset2 = offset

                            if offset< 0 :
                                offset2 = -offset2
                                im_new_padded= np.pad(im2,((0,0), (offset2,0)), 'constant')

                                col_padding = int(im1.shape[1] -im_new_padded.shape[1])

                                if col_padding <0 :
                                    im1= np.pad(im1,((0,0), (0,-col_padding)), 'constant')

                                else: 
                                    im_new_padded= np.pad(im_new_padded,((0,0), (0,col_padding)), 'constant')

                            else:
                                im_new_padded= np.pad(im2[:,offset2:],((0,0), (0,col_padding+offset2)), 'constant')  


                            #print('size_change',im_new_padded.shape)

                            im_new_padded = im_new_padded[remove_rows:, : ]
                            img_stitched = np.concatenate((im1[:-1,:],im_new_padded), axis=0)


                        im_rot = img_stitched

                        #print('new_Size', im_rot.shape)


                        #new folder with joint_name for images that were stitched
                        outfile_name = new_joint_folder +'_'+ channel_name    
                        #ad extra padding to make the new stitched image the same size as all other images
                        img_full_size = change_img_size_singleTiffs(im_rot, max_dimensions)                    
                        #save single tiff with channel name addedcsv_pannel_metal
                        img_full_size = np.array(img_full_size, dtype='uint16')
                        imsave(os.path.join(singleTiff_folder_name, outfile_name), img_full_size, compress=False)

            else:
                print('Check dimensions of the images, different number of channels')

        alredy_stitched.append(key)
        alredy_stitched.append(key2)

    else:
        i_name = str(images[key])
        img_ori = imread(os.path.join(img_folder_initial,i_name))
        
        #print('new_Size_not_modified', img_ori.shape)
        
        i_name_new = remove_str_from_end(i_name, '.tiff')
        i_name_new = i_name_new.replace(" ", "_")
        singleTiff_folder_name = os.path.join(single_tiff_folder,i_name_new)   

        if not os.path.exists(singleTiff_folder_name):
            os.mkdir(singleTiff_folder_name)

        for i in range(6, img_ori.shape[0]):
            
            channel_name = str(channels[i])
            if channel_name in selected_metals:

        
                if key == 60:               
                    im = img_ori[i,:,:]
                    im = rotateImage(im, 180)

                else:     
                    # new_single_tiff_name = im_name + channel_name               
                    im = img_ori[i,:,:]

                i_full_size = change_img_size_singleTiffs(im, max_dimensions)
                outfile_name = i_name_new +'_'+ channel_name
                #save single tiff with channel name addedcsv_pannel_metal
                i_full_size = np.array(i_full_size, dtype='uint16')
                imsave(os.path.join(singleTiff_folder_name, outfile_name), i_full_size, compress=False)


In [ ]:
channel = 'Ir191'
channel_stack = get_image_stack_for_one_channel(single_tiff_folder, channel, final3d_order)
img_check = imread(channel_stack, pattern = None)

In [ ]:
channel2 = 'Nd148'
channel_stack2 = get_image_stack_for_one_channel(single_tiff_folder, channel2, final3d_order)
img_check2 = imread(channel_stack2, pattern = None)

You can uncomment the second command 'viewr.add_image' to view mutliple channels for the 3D stack:

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(img_check, name = channel,contrast_limits= [0,50],colormap='cyan')
    viewer.add_image(img_check2, name =channel2,contrast_limits= [0,50],colormap='yellow')

End of notebook